In [1]:
import d2lzh as d2l
from torch.nn import init
from torch import nn
import torch
from torch import optim
from torch.nn import functional as F

def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    block = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1),
        nn.ReLU(),
    )
    for layer in block:
        if type(layer) is nn.Conv2d:
            init.xavier_normal_(layer.weight)
    return block

In [2]:
class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()
    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.shape[2:])

In [3]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, stride=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, stride=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(),
    nin_block(384, 10, kernel_size=3, stride=1, padding=1),
    GlobalAvgPool2d(),
    d2l.FlattenLayer()
)

In [4]:
X = torch.rand((1, 1, 224, 224))
for name, layer in net.named_children():
    X = layer(X)
    print(name, 'output shape:\t', X.shape)

0 output shape:	 torch.Size([1, 96, 54, 54])
1 output shape:	 torch.Size([1, 96, 26, 26])
2 output shape:	 torch.Size([1, 256, 26, 26])
3 output shape:	 torch.Size([1, 256, 12, 12])
4 output shape:	 torch.Size([1, 384, 12, 12])
5 output shape:	 torch.Size([1, 384, 5, 5])
6 output shape:	 torch.Size([1, 384, 5, 5])
7 output shape:	 torch.Size([1, 10, 5, 5])
8 output shape:	 torch.Size([1, 10, 1, 1])
9 output shape:	 torch.Size([1, 10])


In [5]:
lr, num_epochs, batch_size = 0.1, 5, 128
device = 'cuda' if torch.cuda.is_available() else 'cpu'
optimizer = optim.SGD(net.parameters(), lr=lr)
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device,
              num_epochs)

training on cuda
epoch 1, loss 0.0180, train acc 0.129, test acc 0.188, time 40.7 sec
epoch 2, loss 0.0173, train acc 0.207, test acc 0.201, time 39.7 sec


Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/nick/miniconda3/envs/pytorch/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/nick/miniconda3/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/nick/miniconda3/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/nick/miniconda3/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/home/nick/miniconda3/envs/pytorch/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/nick/miniconda3/envs/pytorch/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/nick/miniconda3/envs/pytor

KeyboardInterrupt: 